# 5장. 케라스2로 구현하는 RNN(순환신경망)
## 5.2 문장을 판별하는 LSTM 구현 
LSTM을 이용하여 문장의 의미를 이해하는 예제를 구현

In [ ]:
# set to use CPU
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

### 5.2.1 라이브러리 패키지 임포트
1. LSTM을 이용한 판별망 구현에 필요한 라이브러리 임포트

In [ ]:
from __future__ import print_function

In [ ]:
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras import layers, models

### 5.2.2 데이터 준비     
2. 데이터는 케라스가 제공하는 공개 데이터인 IMDB를 사용합니다

In [ ]:
class Data:
    def __init__(self, max_features=20000, maxlen=80):
        (self.x_train, self.y_train), (self.x_test, self.y_test) = \
            imdb.load_data(num_words=max_features)
        self.x_train = sequence.pad_sequences(self.x_train, maxlen=maxlen)
        self.x_test = sequence.pad_sequences(self.x_test, maxlen=maxlen)

### 5.2.3 모델링 
3. LSTM 모델링을 위한 클래스를 선언

In [ ]:
class RNN_LSTM(models.Model):
    def __init__(self, max_features, maxlen):
        x = layers.Input((maxlen,))
        h = layers.Embedding(max_features, 128)(x)
        h = layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)(h)
        y = layers.Dense(1, activation='sigmoid')(h)
        super().__init__(x, y)

        # try using different optimizers and different optimizer configs
        self.compile(loss='binary_crossentropy',
                    optimizer='adam', metrics=['accuracy'])

### 5.2.4 학습 및 성능 평가 
4. 학습 및 성능 평가를 담당할 머신 클래스

In [ ]:
class Machine:
    def __init__(self,
                max_features=20000,
                maxlen=80):
        self.data = Data(max_features, maxlen)
        self.model = RNN_LSTM(max_features, maxlen)

    def run(self, epochs=3, batch_size=32):
        data = self.data
        model = self.model
        print('Training stage')
        print('==============')
        model.fit(data.x_train, data.y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 validation_data=(data.x_test, data.y_test))

        score, acc = model.evaluate(data.x_test, data.y_test,
                                   batch_size=batch_size)
        print('Test performance: accuracy={0}, loss={1}'.format(acc, score))

In [ ]:
m = Machine()
m.run()

---
### 전체 코드

In [ ]:
# ① 라이브러리 임포트
from __future__ import print_function
from keras.preprocessing import sequence
from keras.datasets import imdb
from keras import layers, models

# ② 데이터 준비 
class Data:
    def __init__(self, max_features=20000, maxlen=80):
        (self.x_train, self.y_train), (self.x_test, self.y_test) = imdb.load_data(num_words=max_features)
        self.x_train = sequence.pad_sequences(self.x_train, maxlen=maxlen)
        self.x_test = sequence.pad_sequences(self.x_test, maxlen=maxlen)

# ③ 모델링
class RNN_LSTM(models.Model):
    def __init__(self, max_features, maxlen):
        x = layers.Input((maxlen,))
        h = layers.Embedding(max_features, 128)(x)
        h = layers.LSTM(128, dropout=0.2, recurrent_dropout=0.2)(h)
        y = layers.Dense(1, activation='sigmoid')(h)
        super().__init__(x, y)

        # try using different optimizers and different optimizer configs
        self.compile(loss='binary_crossentropy',
                    optimizer='adam', metrics=['accuracy'])

# ④ 학습 및 성능 평가 
class Machine:
    def __init__(self,
                max_features=20000,
                maxlen=80):
        self.data = Data(max_features, maxlen)
        self.model = RNN_LSTM(max_features, maxlen)

    def run(self, epochs=3, batch_size=32):
        data = self.data
        model = self.model
        print('Training stage')
        print('==============')
        model.fit(data.x_train, data.y_train,
                 batch_size=batch_size,
                 epochs=epochs,
                 validation_data=(data.x_test, data.y_test))

        score, acc = model.evaluate(data.x_test, data.y_test,
                                   batch_size=batch_size)
        print('Test performance: accuracy={0}, loss={1}'.format(acc, score))

def main():
    m = Machine()
    m.run()

main()